# Lab 10: Linear programming

In this lab, we will use Linear programming to solve some problems.
Differently from the previous lab, in this one, you will not ask to implement some part of the linear programming paradigm.
Instead, you will ask to translate the problems from the "natural" language to the standard (slack) form.
Moreover, for exercises 1 and 2 consider plotting the solution space and highlighting the different components. 

In the following (hidden) block, the utilities used for running the experiments are implemented.

In [ ]:
from scipy.optimize import linprog, OptimizeResult
import numpy as np
import matplotlib.pyplot as plt

# Ex. 0

A large factory makes tables and chairs.
Each table returns a profit of 200 EUR and each chair a profit of 100 EUR.
Each table takes 1 unit of metal and 3 units of wood and each chair takes 2 units
of metal and 1 unit of wood. The factory has 600 units of metal and 900 units of wood.


---

Slack form:

maximize  $2x_1 + x_2$

subject to 

- $3x_1 + x_2 ≤ 9$
- $x_1 + 2x_2 ≤ 6$
- $x_1, x_2 ≥ 0$

In [ ]:
c = np.array([2, 1])
G = np.array([[3, 1],
              [1, 2]])
h = np.array([9, 6])
r: OptimizeResult = linprog(
    c=-c,
    A_ub=G,
    b_ub=h,
    bounds=[(0, None), (0, None)],
    method='simplex'
)

print(r)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)


def y1(x: float) -> float:
    return 9 - 3 * x


def y2(x: float) -> float:
    return (6 - x) / 2


x, y = np.meshgrid(np.linspace(0, 20, 5000), np.linspace(0, 20, 5000))

feasible = (
        (3 * x + y <= 9) &
        (x + 2 * y <= 6) &
        (x >= 0) &
        (y >= 0)
)

for ax in [ax1, ax2]:
    ax.imshow(
        feasible.astype(int),
        extent=(x.min(), x.max(), y.min(), y.max()),
        origin="lower",
        cmap="Greys",
        alpha=0.2
    )

    xs = np.asarray([-100, 100])

    ax.plot(xs, y1(xs), c='C1', label='$3x + y \leq 9$')
    ax.plot(xs, y2(xs), c='C2', label='$x + 2y \leq 6$')
    ax.axline(r.x, slope=-2, c='C4', linestyle='dotted', label='$2x + y$')
    ax.axvline(0)
    ax.axhline(0)

    ax.scatter(r.x[0], r.x[1], marker=(5, 2), c='r', alpha=0.8, s=100)

ax1.set(
    xlim=[-5, 10],
    ylim=[-5, 10]
)

ax2.set(
    xlim=[-2, 5],
    ylim=[-2, 5]
)

handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center')

fig.savefig("0.svg")

# Ex. 1
A company makes two products (X and Y) using two machines (A and B).
Each unit of X that is produced requires 50 minutes processing time on machine A and 30 minutes processing time on machine B.
Each unit of Y that is produced requires 24 minutes processing time on machine A and 33 minutes processing time on machine B.

At the start of the current week there are 30 units of X and 90 units of Y in stock.
Available processing time on machine A is forecast to be 40 hours and on machine B is forecast to be 35 hours.

The demand for X in the current week is forecast to be 75 units and for Y is forecast to be 95 units.
Company policy is to maximise the combined sum of the units of X and the units of Y in stock at the end of the week.

---

Slack form:

maximize  $#X + #Y$

subject to

- $#X (50m) + #Y (24m) \leq 40h$
- $#X (30m) + #Y (33m) \leq 35h$
- $#X > 75 - 30$
- $#Y > 95 - 90$

the result is the number of units produced, from which I have to subtract the number of units sold

In [ ]:
c = np.array([1, 1])
G = np.array([[50, 24],
              [30, 33]])
h = np.array([40, 35]) * 60
r: OptimizeResult = linprog(
    c=-c,
    A_ub=G,
    b_ub=h,
    bounds=[(75 - 30, None), (95 - 90, None)],
    method='simplex'
)
print(r)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)


def y1(x: float) -> float:
    return (40 * 60 - 50 * x) / 24


def y2(x: float) -> float:
    return (35 * 60 - 30 * x) / 33


x, y = np.meshgrid(np.linspace(44, 46, 5000), np.linspace(4, 8, 5000))

feasible = (
        (50 * x + 24 * y <= 40 * 60) &
        (30 * x + 33 * y <= 35 * 60) &
        (x > 75 - 30) &
        (y > 95 - 90)
)

for ax in [ax1, ax2]:
    ax.imshow(
        feasible.astype(int),
        extent=(x.min(), x.max(), y.min(), y.max()),
        origin="lower",
        cmap="Greys",
        alpha=0.2
    )

    xs = np.asarray([-100, 100])

    ax.plot(xs, y1(xs), c='C1', label='$50x + 24y \leq 40 * 60$')
    ax.plot(xs, y2(xs), c='C2', label='$30x + 33y \leq 35 * 60$')
    ax.axline(r.x, slope=-1, c='C4', linestyle='dotted', label='$x + y$')
    ax.axvline(75 - 30)
    ax.axhline(95 - 90)

    ax.scatter(r.x[0], r.x[1], marker=(5, 2), c='r', alpha=0.8, s=100)

ax1.set(
    xlim=[25, 70],
    ylim=[0, 45]
)

ax2.set(
    xlim=[43.5, 47.5],
    ylim=[4, 8]
)

handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center')

fig.savefig("1.svg")

# Ex. 2
A factory manufactures chairs and tables, each requiring the use of three operations: Cutting, Assembly, and Finishing.
The first operation can be used at most 40 hours;
the second at most 42 hours;
and the third at most 25 hours.

A chair requires 1 hour of cutting, 2 hours of assembly, and 1 hour of finishing;
a table needs 2 hours of cutting, 1 hour of assembly, and 1 hour of finishing.

If the profit is 20 per unit for a chair and 30 for a table, how many units of each should be manufactured to maximize profit?

---

Slack form:

maximize $20 #C + 30 #T$

subject to:

- $#C(1h) + #T(2h) \leq 40h$ cutting
- $#C(2h) + #T(1h) \leq 42h$ assembling
- $#C(1h) + #T(1h) \leq 25h$ finishing
- $#C, #T \geq 0$

In [ ]:
c = np.array([20, 30])
G = np.array([[1, 2],
              [2, 1],
              [1, 1]])
h = np.array([40, 42, 25])
r: OptimizeResult = linprog(
    c=-c,
    A_ub=G,
    b_ub=h,
    bounds=[(0, None), (0, None)],
    method='simplex'
)
print(r)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)


def y1(x: float) -> float:
    return (40 - x) / 2


def y2(x: float) -> float:
    return 42 - 2 * x


def y3(x: float) -> float:
    return 25 - x


x, y = np.meshgrid(np.linspace(0, 25, 5000), np.linspace(0, 25, 5000))

feasible = (
        (x + 2 * y <= 40) &
        (2 * x + y <= 42) &
        (x + y <= 25) &
        (x >= 0) &
        (y >= 0)
)

for ax in [ax1, ax2]:
    ax.imshow(
        feasible.astype(int),
        extent=(x.min(), x.max(), y.min(), y.max()),
        origin="lower",
        cmap="Greys",
        alpha=0.2
    )

    xs = np.asarray([-100, 100])

    ax.plot(xs, y1(xs), c='C1', label='$x + 2y \leq 40$')
    ax.plot(xs, y2(xs), c='C2', label='$2x + y \leq 42$')
    ax.plot(xs, y3(xs), c='C3', label='$x + y \leq 25$')
    ax.axline(r.x, slope=-2 / 3, c='C4', linestyle='dotted', label='$20x + 30y$')
    ax.axvline(0)
    ax.axhline(0)

    ax.scatter(r.x[0], r.x[1], marker=(5, 2), c='r', alpha=0.8, s=100)

ax1.set(
    xlim=[-5 - 20, 45 + 10],
    ylim=[-5 - 20, 45 + 10]
)

ax2.set(
    xlim=[-5, 25],
    ylim=[-5, 25]
)

handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center')

fig.savefig("2.svg")

# Ex. 3
We now consider a problem posed and solved by [Hu18].

A mutual fund has USD 100'000 to be invested over a three year horizon.

Three investment options are available:

Annuity:
the fund can pay a same amount of new capital at the beginning of each of three years and receive a payoff of 130% of total capital invested at the end of the third year.
Once the mutual fund decides to invest in this annuity, it has to keep investing in all subsequent years in the three year horizon.

Bank account:
the fund can deposit any amount into a bank at the beginning of each year and receive its capital plus 6% interest at the end of that year.
In addition, the mutual fund is permitted to borrow no more than USD 20,000 at the beginning of each year and is asked to pay back the amount borrowed plus 6% interest at the end of the year.
The mutual fund can choose whether to deposit or borrow at the beginning of each year.

Corporate bond:
At the beginning of the second year, a corporate bond becomes available.
The fund can buy an amount that is no more than USD 50,000 of this bond at the beginning of the second year and at the end of the third year receive a payout of 130% of the amount invested in the bond.

The mutual fund’s objective is to maximize total payout that it owns at the end of the third year.

- Hu18 Y. Hu, Y. & Guo. Operations research. Tsinghua University Press, 5th edition, 2018.

In [ ]:
c = np.array([1.3 * 3, 0, 0, 1.06, 1.30])
G = np.array([[1,     1,     0, 0, 0],
              [1, -1.06,     1, 0, 1],
              [1,     0, -1.06, 1, 0],

              [-1,     -1,      0,  0,  0],
              [-1,   1.06,     -1,  0, -1],
              [-1,     -0,   1.06, -1,  0],
              ])
h = np.array([100e3, 0, 0, -100e3, 0, 0])
r: OptimizeResult = linprog(
    c=-c,
    A_ub=G,
    b_ub=h,
    bounds=[(0, None), (-20e3, None), (-20e3, None), (-20e3, None), (0, 50e3)],
    method='simplex'
)